# 第3章 保存恢复模型
## 3.1 保存模型  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;在构造期末尾创建一个 saver 节点，在执行期中调用save()方法，传入一个会话和检查点文件的路径即可。以保存加州房价线性模型为例：

In [8]:
# 导入相关包
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import datetime

In [9]:
# 下载及整理数据
housing = fetch_california_housing()
m,n = housing.data.shape # 获取数据的行列数
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data] # 为数据添加偏差项，即添加y=ax+b中的b

In [10]:
# 数据预处理
scaler = StandardScaler().fit(housing_data_plus_bias)
scaled_housing_data_plus_bias = scaler.transform(housing_data_plus_bias)

In [11]:
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
n_epochs = 1000
batch_size = 100
n_batches= int(np.ceil(m/batch_size))
global_learning_rate = 0.01
XT = tf.transpose(X)
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")     # * 参数 seed=42
y_pred =  tf.matmul(X, theta, name="prediction")                          # 预测值
error = y_pred-y                                                          # 误差
mse = tf.reduce_mean(tf.square(error), name="mse")                        # 均方误差(成本函数)                                 
# 调用特定的优化器求解梯度并优化
optimizer = tf.train.GradientDescentOptimizer(learning_rate = global_learning_rate)
training_op = optimizer.minimize(mse)
# 添加一个saver节点用来保存模型参数
saver = tf.train.Saver()

In [7]:
# 执行阶段
init = tf.global_variables_initializer()

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index) 
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices] 
    y_batch = housing.target.reshape(-1, 1)[indices] 
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch%100 == 0:
            save_path =  saver.save(sess, "./tmp/my_model.ckpt")
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
    best_theta = theta.eval()
    print("The best theta is", best_theta)
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")

The best theta is [[ 0.6356673 ]
 [ 0.8377844 ]
 [ 0.10645497]
 [-0.25947902]
 [ 0.29196444]
 [ 0.00181689]
 [ 0.2128084 ]
 [-0.89034677]
 [-0.85242176]]
